# Detección de tópicos por tweet en threads

## Importacion de librerias y definicion de funciones

Se importan las librerias necesarias y se definen las funciones para tokenizar, lemmatizar y para preparar el texto para el LDA.

En la tokenizacion se eliminan los hashtags y los usuarios citados

In [13]:
import os
import pymongo

In [14]:
#conexion mongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["twitter-memoria"]

csv_all = mydb["csv_all"]

In [15]:
import spacy
import re
spacy.load('en')

from spacy.lang.en import English
parser = English()

import nltk

nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    """indexes = [m.span() for m in re.finditer(r'#\w+',text,flags=re.IGNORECASE)]
    for start,end in indexes:
        tokens.merge(start_idx=start,end_idx=end)"""
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            continue
        elif token.orth_.startswith('#'):
            #lda_tokens.append(token)
            continue
        elif token.orth_.startswith('@'):
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

tokenize("esto es un #ejemplo @usuario")


['esto', 'es', 'un', 'ejemplo']

## Importacion de archivos

Se cargan los archivos csv y se agrupan los tweets por threads, para luego crear un diccionario de tweets por cada thread (thread 1 : tweet1, tweet2...)

In [17]:
import random
import pandas as pd


In [18]:
string = '\n'

hilos_ref = []
all_hilos = []
tweets = csv_all.find({})

threads1 = {}
threads2 = {}
threads3 = {}
threads4 = {}
threads5 = {}
kthreads1 = {}
kthreads2 = {}
kthreads3 = {}
kthreads4 = {}
kthreads5 = {}

for tweet in tweets:
    if tweet["hilo_ref"] not in hilos_ref:
        hilos_ref.append(tweet["hilo_ref"])

for hilo in hilos_ref:
    tweets_hilo = csv_all.find({"hilo_ref": hilo})
    lista_aux_hilos = []
    csv = ""
    for tweet_aux in tweets_hilo:
        lista_aux_hilos.append(tweet_aux["text"])
        csv = tweet_aux["csv"]
        hilo_csv = tweet_aux["hilo"]
    
    if csv == "csv1":
        threads1[hilo_csv] = lista_aux_hilos
        kthreads1[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv2":
        threads2[hilo_csv] = lista_aux_hilos
        kthreads2[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv3":
        threads3[hilo_csv] = lista_aux_hilos
        kthreads3[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv4":
        threads4[hilo_csv] = lista_aux_hilos
        kthreads4[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv5":
        threads5[hilo_csv] = lista_aux_hilos
        kthreads5[hilo_csv] = string.join(lista_aux_hilos)
        
Tthreads1 = list(kthreads1.values())
Tthreads2 = list(kthreads2.values())
Tthreads3 = list(kthreads3.values())
Tthreads4 = list(kthreads4.values())
Tthreads5 = list(kthreads5.values())


## Creación de diccionario de tweets por threads

Se agruparán los tweets de cada hilo en un diccionario para cada archivo.

In [19]:
threads1

{'Thread 1': ['Extraordinary evidence at Treasury committee from Jon Thompson, CEO of HMRC on customs and Brexit today https://t.co/DJhIQhmVwJ',
  "The Brexiter favourite Max Fac - would cost business between £17 and £20bn a year\n\n- that's almost 1% of GDP\n\n- just for filling in forms\n\nThanks #Brexit",
  '"We think we can manage the risk - we think we can" he said. He didn\'t sound so sure. \n\nAnd "the potential backdoor risk applies to both models" he added\n\nDidn\'t sound like officials think either is sensible',
  'Mr Thompson said he did not expect the EU to reciprocate over the customs partnership. \n\nWhat that means is UK collects tariffs for EU and hands it over when a ship lands in Felixtowe and drives to Calais, but if ship first lands in Rotterdam, EU keeps the import tariffs.',
  'Both would not be ready by 2021. Max Fac needs 3 years. Customs Partnership requires 5, Mr Thompson said.\n\nThe border would be "functioning", but if technology not ready ministers would 

## LDA para cada thread de cada CSV

Se definen la cantidad de topicos a detectar, en conjunto con la cantidad de palabras que se mostraran al imprimir los topicos detectados.

La detección de tópicos se realizará a cada thread de todos los archivos CSV, por lo que se considerará cada tweet del thread como un documento.

In [20]:
import gensim
from gensim import corpora
NUM_TOPICS = 5
NUM_WORDS = 5
import pickle

### CSV five_ten

In [21]:
THIS_FOLDER = os.getcwd()
threads_leer = threads1
carpeta_guardar = "tpcsv1"

#Poblar text_data

for hilos in threads_leer:
    camino = os.path.join(THIS_FOLDER, carpeta_guardar)
    text_data = []
    documentos = []
    dictionary = []
    corpus = []
    print(hilos)
    documentos = threads_leer[hilos]

    #print(documentos)

    for line in documentos:
        #print(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .009:
            #print(tokens)
            text_data.append(tokens)

    #print(text_data) 
    NDIC = camino+"\\"+hilos+"_t_dictionary1.gensim"
    NMOD = camino+"\\"+hilos+"_t_model1.gensim"
    NCOR = camino+"\\"+hilos+"_t_corpus1.pkl"
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open(NCOR, 'wb'))
    dictionary.save(NDIC)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(NMOD)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    for topic in topics:
        print(topic)

Thread 10
(0, '0.028*"create" + 0.028*"make" + 0.027*"free" + 0.027*"expensive" + 0.015*"asylum"')
(1, '0.006*"immigration" + 0.006*"invest" + 0.006*"allow" + 0.006*"resource" + 0.006*"money"')
(2, '0.027*"green" + 0.014*"immig" + 0.014*"deportation" + 0.014*"judges" + 0.014*"give"')
(3, '0.022*"create" + 0.022*"visa" + 0.022*"base" + 0.022*"border" + 0.022*"gutting"')
(4, '0.036*"immigration" + 0.020*"give" + 0.020*"judges" + 0.020*"immig" + 0.020*"fine"')
Thread 14
(0, '0.071*"internetbillofrights" + 0.071*"internetbillofrightsnow" + 0.071*"oigreport" + 0.071*"qanon" + 0.071*"greatawakening"')
(1, '0.057*"thestorm" + 0.057*"greatawakening" + 0.057*"qanon" + 0.057*"oigreport" + 0.057*"internetbillofrightsnow"')
(2, '0.060*"hold" + 0.060*"good" + 0.060*"future" + 0.060*"oigreport" + 0.060*"qanon"')
(3, '0.063*"narrative" + 0.063*"operations" + 0.063*"drop" + 0.063*"mockingbirdmedia" + 0.063*"clown"')
(4, '0.063*"greatawakening" + 0.063*"internetbillofrightsnow" + 0.063*"thestorm" + 0.0

(4, '0.042*"cause" + 0.042*"child" + 0.023*"bootpruitt" + 0.023*"common" + 0.023*"show"')
Thread 19
(0, '0.031*"american" + 0.031*"influence" + 0.031*"actor" + 0.031*"arrest" + 0.017*"trump"')
(1, '0.047*"government" + 0.036*"deepstate" + 0.036*"design" + 0.025*"deep" + 0.025*"state"')
(2, '0.007*"american" + 0.007*"destroy" + 0.007*"october" + 0.007*"nuclear" + 0.007*"deepstate"')
(3, '0.033*"help" + 0.033*"need" + 0.033*"vote" + 0.018*"missile" + 0.018*"nuclear"')
(4, '0.078*"pray" + 0.033*"deepstate" + 0.033*"must" + 0.018*"going" + 0.018*"make"')
Thread 22
(0, '0.028*"unresolved" + 0.028*"expert" + 0.028*"feel" + 0.028*"scores" + 0.028*"attack"')
(1, '0.052*"would" + 0.035*"mkultra" + 0.035*"world" + 0.019*"feel" + 0.019*"helpless"')
(2, '0.027*"expert" + 0.027*"qanon" + 0.027*"extreme" + 0.027*"pretending" + 0.027*"adrenalin"')
(3, '0.025*"complicate" + 0.025*"segment" + 0.025*"answer" + 0.025*"look" + 0.025*"mkultra"')
(4, '0.009*"magic" + 0.009*"segment" + 0.009*"shadow" + 0.009

(1, '0.025*"meaningful" + 0.025*"felt" + 0.025*"year" + 0.025*"oscar" + 0.025*"critic"')
(2, '0.033*"diversity" + 0.018*"advocate" + 0.018*"success" + 0.018*"white" + 0.018*"work"')
(3, '0.007*"reject" + 0.007*"initially" + 0.007*"academy" + 0.007*"behalf" + 0.007*"latina"')
(4, '0.007*"critic" + 0.007*"addendum" + 0.007*"compel" + 0.007*"look" + 0.007*"latino"')
Thread 43
(0, '0.050*"kind" + 0.050*"patriot" + 0.050*"real" + 0.050*"remember" + 0.050*"help"')
(1, '0.041*"qanon" + 0.041*"speak" + 0.041*"hero" + 0.041*"wethepeopledemandjustice" + 0.041*"releasethememo"')
(2, '0.040*"upon" + 0.040*"seizure" + 0.040*"fighting" + 0.040*"light" + 0.040*"child"')
(3, '0.046*"undermine" + 0.046*"trend" + 0.046*"shut" + 0.046*"president" + 0.046*"election"')
(4, '0.046*"like" + 0.046*"need" + 0.046*"antifa" + 0.046*"force" + 0.046*"matter"')
Thread 41
(0, '0.096*"trump" + 0.044*"spend" + 0.033*"million" + 0.033*"campaign" + 0.033*"meeting"')
(1, '0.041*"russia" + 0.041*"connection" + 0.041*"medi

(0, '0.045*"tech" + 0.045*"pathway" + 0.024*"idea" + 0.024*"innate" + 0.024*"contribute"')
(1, '0.048*"country" + 0.048*"know" + 0.048*"female" + 0.048*"dominate" + 0.026*"also"')
(2, '0.038*"job" + 0.038*"article" + 0.038*"woman" + 0.038*"tech" + 0.021*"pipeline"')
(3, '0.009*"article" + 0.009*"ask" + 0.009*"tech" + 0.009*"give" + 0.009*"links"')
(4, '0.082*"woman" + 0.056*"tech" + 0.043*"need" + 0.029*"structural" + 0.029*"reason"')
Thread 72
(0, '0.091*"republicana" + 0.091*"gran" + 0.091*"gràcies" + 0.091*"twitter" + 0.091*"censura"')
(1, '0.091*"teixint" + 0.091*"història" + 0.091*"estem" + 0.091*"seguiu" + 0.091*"twitter"')
(2, '0.091*"gran" + 0.091*"republicana" + 0.091*"xarxa" + 0.091*"atenció" + 0.091*"gràcies"')
(3, '0.091*"republicana" + 0.091*"estem" + 0.091*"gràcies" + 0.091*"gran" + 0.091*"xarxa"')
(4, '0.091*"republicana" + 0.091*"gràcies" + 0.091*"xarxa" + 0.091*"censura" + 0.091*"atenció"')
Thread 67
(0, '0.048*"truth" + 0.033*"cabal" + 0.018*"involve" + 0.018*"arrest"

(1, '0.148*"know" + 0.029*"thestorm" + 0.029*"qanon" + 0.029*"friend" + 0.029*"greatawakening"')
(2, '0.045*"greatawakening" + 0.045*"thestorm" + 0.045*"qanon" + 0.045*"think" + 0.025*"pilling"')
(3, '0.033*"around" + 0.033*"uneasy" + 0.033*"mother" + 0.033*"lifetime" + 0.033*"revolution"')
(4, '0.064*"please" + 0.064*"unroll" + 0.011*"stuff" + 0.011*"share" + 0.011*"story"')
Thread 94
(0, '0.034*"microtargeting" + 0.019*"weapon" + 0.019*"getting" + 0.019*"misinformation" + 0.019*"difference"')
(1, '0.040*"military" + 0.040*"cambridgeanalytica" + 0.040*"invest" + 0.022*"bannon" + 0.022*"create"')
(2, '0.026*"campaign" + 0.026*"information" + 0.026*"cambridgeanalytica" + 0.026*"night" + 0.026*"last"')
(3, '0.047*"dark" + 0.047*"even" + 0.026*"fake" + 0.026*"news" + 0.026*"target"')
(4, '0.037*"population" + 0.025*"information" + 0.025*"microtargeting" + 0.025*"cambridgeanalytica" + 0.025*"obama"')
Thread 96
(0, '0.038*"cambridge" + 0.038*"target" + 0.038*"voter" + 0.038*"analytica" + 0.

### CSV ten_fifteen

In [22]:
THIS_FOLDER = os.getcwd()
threads_leer = threads2
carpeta_guardar = "tpcsv2"

#Poblar text_data

for hilos in threads_leer:
    camino = os.path.join(THIS_FOLDER, carpeta_guardar)
    text_data = []
    documentos = []
    dictionary = []
    corpus = []
    print(hilos)
    documentos = threads_leer[hilos]

    #print(documentos)

    for line in documentos:
        #print(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .009:
            #print(tokens)
            text_data.append(tokens)

    #print(text_data) 
    NDIC = camino+"\\"+hilos+"_t_dictionary1.gensim"
    NMOD = camino+"\\"+hilos+"_t_model1.gensim"
    NCOR = camino+"\\"+hilos+"_t_corpus1.pkl"
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open(NCOR, 'wb'))
    dictionary.save(NDIC)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(NMOD)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    for topic in topics:
        print(topic)

Thread 3
(0, '0.045*"human" + 0.038*"wall" + 0.030*"tunnel" + 0.030*"probability" + 0.023*"part"')
(1, '0.070*"electron" + 0.048*"back" + 0.048*"ball" + 0.026*"like" + 0.026*"passing"')
(2, '0.009*"saturdaynightthoughts" + 0.009*"like" + 0.009*"passing" + 0.009*"probability" + 0.009*"might"')
(3, '0.009*"saturdaynightthoughts" + 0.009*"like" + 0.009*"passing" + 0.009*"probability" + 0.009*"begin"')
(4, '0.072*"barrier" + 0.055*"probability" + 0.055*"electron" + 0.038*"particle" + 0.038*"using"')
Thread 2
(0, '0.032*"definition" + 0.032*"away" + 0.032*"solution" + 0.017*"racist" + 0.017*"lose"')
(1, '0.049*"stuff" + 0.027*"mean" + 0.027*"people" + 0.027*"whole" + 0.027*"politics"')
(2, '0.048*"folks" + 0.029*"racism" + 0.020*"really" + 0.020*"need" + 0.020*"happen"')
(3, '0.030*"racial" + 0.021*"moral" + 0.021*"racism" + 0.021*"exploitation" + 0.021*"cultural"')
(4, '0.029*"racist" + 0.029*"greatest" + 0.029*"words" + 0.029*"4/13" + 0.029*"accomplishment"')
Thread 6
(0, '0.138*"page" + 

(1, '0.050*"putin" + 0.050*"trump" + 0.035*"agalarov" + 0.035*"say" + 0.019*"would"')
(2, '0.056*"trump" + 0.030*"mueller" + 0.030*"clear" + 0.030*"trumprussia" + 0.030*"laser"')
(3, '0.067*"trump" + 0.051*"putin" + 0.035*"know" + 0.035*"dowd" + 0.035*"letter"')
(4, '0.045*"pageant" + 0.045*"putin" + 0.045*"trump" + 0.031*"attend" + 0.031*"miss"')
Thread 17
(0, '0.009*"things" + 0.009*"remember" + 0.009*"movie" + 0.009*"climax" + 0.009*"greatest"')
(1, '0.045*"know" + 0.025*"greatawakening" + 0.025*"releasethememo" + 0.025*"memoday" + 0.025*"qanon"')
(2, '0.074*"thestorm" + 0.068*"qanon" + 0.068*"memoday" + 0.068*"releasethememo" + 0.068*"greatawakening"')
(3, '0.051*"please" + 0.051*"unroll" + 0.009*"time" + 0.009*"mueller" + 0.009*"memoday"')
(4, '0.009*"things" + 0.009*"remember" + 0.009*"movie" + 0.009*"climax" + 0.009*"greatest"')
Thread 23
(0, '0.042*"prince" + 0.042*"nader" + 0.042*"btwn" + 0.032*"backchannel" + 0.032*"trump"')
(1, '0.036*"meeting" + 0.028*"kush" + 0.028*"seyche

(3, '0.040*"eric" + 0.027*"investor" + 0.027*"spacex" + 0.027*"clinton" + 0.027*"google"')
(4, '0.021*"rocket" + 0.021*"right" + 0.021*"surveillance" + 0.021*"together" + 0.021*"budget"')
Thread 33
(0, '0.019*"election" + 0.019*"news" + 0.019*"ban" + 0.019*"democracy" + 0.013*"racism"')
(1, '0.004*"woman" + 0.004*"public" + 0.004*"accused" + 0.004*"soundbytes" + 0.004*"hold"')
(2, '0.024*"country" + 0.024*"normal" + 0.024*"terms" + 0.024*"look" + 0.013*"attack"')
(3, '0.019*"woman" + 0.010*"good" + 0.010*"anti" + 0.010*"office" + 0.010*"front"')
(4, '0.012*"nation" + 0.012*"tear" + 0.012*"sense" + 0.012*"self" + 0.012*"lie"')
Thread 38
(0, '0.047*"security" + 0.047*"clearance" + 0.047*"know" + 0.047*"pension" + 0.047*"going"')
(1, '0.083*"flynn" + 0.035*"pay" + 0.035*"since" + 0.035*"partner" + 0.019*"russian"')
(2, '0.013*"flynn" + 0.013*"partner" + 0.013*"business" + 0.013*"letter" + 0.013*"security"')
(3, '0.045*"partner" + 0.045*"violate" + 0.045*"request" + 0.045*"respond" + 0.045

(4, '0.033*"semi" + 0.033*"automatic" + 0.033*"hold" + 0.023*"legislation" + 0.023*"weapon"')
Thread 61
(0, '0.044*"pac" + 0.024*"american" + 0.024*"list" + 0.024*"donation" + 0.024*"representative"')
(1, '0.030*"cycle" + 0.016*"receive" + 0.016*"within" + 0.016*"last" + 0.016*"entity"')
(2, '0.134*"party" + 0.119*"️repub" + 0.068*"cmte" + 0.046*"repub" + 0.031*"rebekahmercer"')
(3, '0.030*"american" + 0.016*"action" + 0.016*"ca/#mercers" + 0.016*"tie" + 0.016*"conservative"')
(4, '0.031*"mercer" + 0.031*"️make" + 0.031*"america" + 0.017*"many" + 0.017*"utility"')
Thread 62
(0, '0.027*"employee" + 0.027*"say" + 0.027*"stun" + 0.027*"american" + 0.027*"tell"')
(1, '0.039*"campaign" + 0.027*"employee" + 0.027*"state" + 0.027*"term" + 0.027*"election"')
(2, '0.042*"election" + 0.042*"mercer" + 0.042*"bannon" + 0.023*"cambridgeanalytica" + 0.023*"superpacs"')
(3, '0.048*"campaign" + 0.033*"cambridge" + 0.033*"legal" + 0.033*"make" + 0.033*"illegal"')
(4, '0.035*"election" + 0.024*"cambridg

(4, '0.034*"memoday" + 0.026*"nunesmemo" + 0.026*"know" + 0.018*"really" + 0.018*"temptation"')
Thread 87
(0, '0.016*"technology" + 0.016*"much" + 0.016*"destroy" + 0.016*"company" + 0.015*"challenge"')
(1, '0.039*"control" + 0.027*"people" + 0.027*"information" + 0.015*"draintheswampuk" + 0.015*"destroy"')
(2, '0.030*"swamp" + 0.030*"control" + 0.023*"information" + 0.023*"fight" + 0.016*"people"')
(3, '0.032*"regressiveleft" + 0.032*"culture" + 0.022*"need" + 0.012*"access" + 0.012*"want"')
(4, '0.027*"medium" + 0.027*"thread" + 0.015*"technology" + 0.015*"intelligence" + 0.015*"draintheswampuk"')
Thread 82
(0, '0.026*"move" + 0.018*"release" + 0.018*"memo" + 0.018*"game" + 0.018*"chess"')
(1, '0.027*"rest" + 0.014*"make" + 0.014*"already" + 0.014*"document" + 0.014*"know"')
(2, '0.019*"would" + 0.019*"forward" + 0.019*"memo" + 0.011*"direction" + 0.011*"garbage"')
(3, '0.029*"deepstate" + 0.016*"take" + 0.016*"plan" + 0.016*"need" + 0.016*"draintheswamp"')
(4, '0.027*"qanon" + 0.027

### CSV fifteen_twenty

In [23]:
THIS_FOLDER = os.getcwd()
threads_leer = threads3
carpeta_guardar = "tpcsv3"

#Poblar text_data

for hilos in threads_leer:
    camino = os.path.join(THIS_FOLDER, carpeta_guardar)
    text_data = []
    documentos = []
    dictionary = []
    corpus = []
    print(hilos)
    documentos = threads_leer[hilos]

    #print(documentos)

    for line in documentos:
        #print(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .009:
            #print(tokens)
            text_data.append(tokens)

    #print(text_data) 
    NDIC = camino+"\\"+hilos+"_t_dictionary1.gensim"
    NMOD = camino+"\\"+hilos+"_t_model1.gensim"
    NCOR = camino+"\\"+hilos+"_t_corpus1.pkl"
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open(NCOR, 'wb'))
    dictionary.save(NDIC)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(NMOD)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    for topic in topics:
        print(topic)

Thread 6
(0, '0.167*"host" + 0.103*"bitcoin" + 0.038*"blockchain" + 0.038*"debate" + 0.038*"cash"')
(1, '0.149*"host" + 0.078*"crypto" + 0.043*"hedge" + 0.043*"funds" + 0.043*"matter"')
(2, '0.070*"mention" + 0.070*"honourable" + 0.070*"cryptoharuspex" + 0.070*"another" + 0.012*"honesty"')
(3, '0.038*"best" + 0.038*"episode" + 0.038*"podcast" + 0.038*"crypto" + 0.038*"mention"')
(4, '0.044*"investment" + 0.044*"2017" + 0.044*"bonus" + 0.044*"podcast" + 0.044*"radical"')
Thread 4
(0, '0.032*"tell" + 0.032*"report" + 0.032*"sessions" + 0.032*"indictment" + 0.032*"also"')
(1, '0.052*"thestorm" + 0.052*"qanon" + 0.052*"greatawakening" + 0.049*"internetbillofrights" + 0.045*"enjoytheshow"')
(2, '0.006*"greatawakening" + 0.006*"signature" + 0.006*"enjoytheshow" + 0.006*"relate" + 0.006*"use"')
(3, '0.006*"seal" + 0.006*"podesta" + 0.006*"huma" + 0.006*"radar" + 0.006*"asking"')
(4, '0.041*"internetbillofrights" + 0.041*"thestorm" + 0.041*"greatawakening" + 0.041*"qanon" + 0.032*"enjoytheshow

(1, '0.033*"qanon" + 0.033*"bridge" + 0.018*"tonight" + 0.018*"post" + 0.018*"today"')
(2, '0.040*"qanon" + 0.040*"truth" + 0.031*"kill" + 0.031*"clown" + 0.021*"access"')
(3, '0.046*"qanon" + 0.032*"ask" + 0.032*"remind" + 0.017*"order" + 0.017*"censorship"')
(4, '0.034*"mind" + 0.034*"outside" + 0.034*"control" + 0.034*"qanon" + 0.023*"suspectable"')
Thread 18
(0, '0.028*"bible" + 0.028*"student" + 0.028*"qanon" + 0.028*"fakenewsawards" + 0.019*"show"')
(1, '0.036*"liar" + 0.021*"fact" + 0.021*"assange" + 0.021*"release" + 0.021*"trust"')
(2, '0.036*"qanon" + 0.036*"fakenewsawards" + 0.036*"bible" + 0.036*"student" + 0.019*"shithole"')
(3, '0.045*"qanon" + 0.045*"bible" + 0.045*"fakenewsawards" + 0.045*"student" + 0.020*"president"')
(4, '0.030*"student" + 0.030*"bible" + 0.030*"fakenewsawards" + 0.030*"qanon" + 0.020*"medium"')
Thread 23
(0, '0.048*"immigrant" + 0.030*"unauthorized" + 0.030*"trump" + 0.030*"group" + 0.030*"back"')
(1, '0.026*"people" + 0.026*"identity" + 0.026*"powe

(3, '0.021*"america" + 0.021*"american" + 0.011*"world" + 0.011*"oppress" + 0.011*"fine"')
(4, '0.018*"american" + 0.018*"freedom" + 0.018*"like" + 0.018*"able" + 0.018*"call"')
Thread 35
(0, '0.018*"bannon" + 0.018*"say" + 0.018*"firm" + 0.018*"wylie" + 0.018*"tiny"')
(1, '0.043*"campaign" + 0.016*"separate" + 0.016*"laws" + 0.016*"group" + 0.016*"back"')
(2, '0.025*"though" + 0.025*"supposedly" + 0.025*"receive" + 0.025*"funds" + 0.025*"together"')
(3, '0.060*"leave" + 0.050*"vote" + 0.031*"brexit" + 0.031*"online" + 0.021*"final"')
(4, '0.045*"vote" + 0.040*"brexit" + 0.034*"campaign" + 0.034*"leave" + 0.028*"group"')
Thread 40
(0, '0.028*"company" + 0.028*"handcuff" + 0.015*"need" + 0.015*"time" + 0.015*"people"')
(1, '0.023*"contribution" + 0.023*"personal" + 0.013*"make" + 0.013*"megacorp" + 0.013*"find"')
(2, '0.040*"startup" + 0.027*"company" + 0.014*"show" + 0.014*"join" + 0.010*"megacorp"')
(3, '0.025*"work" + 0.025*"company" + 0.020*"learn" + 0.020*"outside" + 0.020*"startup

(0, '0.038*"former" + 0.014*"look" + 0.014*"totally" + 0.014*"democrat" + 0.014*"start"')
(1, '0.035*"gina" + 0.035*"mccarthy" + 0.027*"scottpruitt" + 0.018*"change" + 0.018*"write"')
(2, '0.019*"former" + 0.019*"record" + 0.019*"mccarthy" + 0.019*"gina" + 0.010*"administrator"')
(3, '0.029*"really" + 0.029*"joke" + 0.029*"thank" + 0.029*"mondaymotivaton" + 0.029*"former"')
(4, '0.022*"former" + 0.022*"job" + 0.022*"slash" + 0.022*"place" + 0.022*"regulation"')
Thread 62
(0, '0.024*"call" + 0.024*"nancy" + 0.024*"executive" + 0.013*"today" + 0.013*"control"')
(1, '0.026*"qanon" + 0.026*"salzman" + 0.026*"clinton" + 0.014*"reveal" + 0.014*"nxivm"')
(2, '0.019*"qanon" + 0.019*"attack" + 0.019*"lock" + 0.010*"tell" + 0.010*"bilk"')
(3, '0.033*"qanon" + 0.023*"attorney" + 0.013*"tell" + 0.013*"ask" + 0.013*"social"')
(4, '0.035*"post" + 0.035*"qanon" + 0.028*"clown" + 0.021*"last" + 0.021*"use"')
Thread 60
(0, '0.018*"المشاركة" + 0.018*"مليون" + 0.012*"المصريين" + 0.012*"ولكن" + 0.012*"الت

(3, '0.043*"tell" + 0.026*"child" + 0.018*"anything" + 0.018*"wife" + 0.018*"come"')
(4, '0.024*"break" + 0.024*"save" + 0.024*"face" + 0.024*"genocide" + 0.024*"world"')
Thread 80
(0, '0.031*"someone" + 0.031*"patience" + 0.017*"process" + 0.017*"overwhelm" + 0.017*"likely"')
(1, '0.041*"understanding" + 0.041*"newfound" + 0.028*"helpful" + 0.028*"part" + 0.028*"take"')
(2, '0.024*"root" + 0.013*"people" + 0.013*"message" + 0.013*"normalize" + 0.013*"increasingly"')
(3, '0.062*"learning" + 0.014*"change" + 0.014*"important" + 0.014*"failure" + 0.014*"refinement"')
(4, '0.036*"professor" + 0.020*"people" + 0.020*"things" + 0.020*"something" + 0.020*"even"')
Thread 84
(0, '0.023*"oathing" + 0.013*"land" + 0.013*"particular" + 0.013*"live" + 0.013*"long"')
(1, '0.024*"people" + 0.016*"jomo" + 0.016*"isolate" + 0.016*"make" + 0.016*"oath"')
(2, '0.025*"tell" + 0.018*"attitude" + 0.018*"colonial" + 0.018*"oath" + 0.018*"take"')
(3, '0.027*"laws" + 0.027*"pass" + 0.018*"oath" + 0.018*"colon

(2, '0.051*"idea" + 0.051*"make" + 0.027*"life" + 0.027*"remixing" + 0.027*"understand"')
(3, '0.078*"writing" + 0.033*"people" + 0.033*"time" + 0.033*"consume" + 0.018*"learning"')
(4, '0.042*"remix" + 0.023*"idea" + 0.023*"free" + 0.023*"people" + 0.023*"exist"')


### CSV twenty_twentyfive

In [24]:
THIS_FOLDER = os.getcwd()
threads_leer = threads4
carpeta_guardar = "tpcsv4"

#Poblar text_data

for hilos in threads_leer:
    camino = os.path.join(THIS_FOLDER, carpeta_guardar)
    text_data = []
    documentos = []
    dictionary = []
    corpus = []
    print(hilos)
    documentos = threads_leer[hilos]

    #print(documentos)

    for line in documentos:
        #print(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .009:
            #print(tokens)
            text_data.append(tokens)

    #print(text_data) 
    NDIC = camino+"\\"+hilos+"_t_dictionary1.gensim"
    NMOD = camino+"\\"+hilos+"_t_model1.gensim"
    NCOR = camino+"\\"+hilos+"_t_corpus1.pkl"
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open(NCOR, 'wb'))
    dictionary.save(NDIC)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(NMOD)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    for topic in topics:
        print(topic)

Thread 2
(0, '0.032*"trade" + 0.022*"policy" + 0.012*"narcissistic" + 0.012*"elite" + 0.012*"liberalisation"')
(1, '0.049*"vote" + 0.026*"leave" + 0.018*"meaningful" + 0.018*"influence" + 0.018*"things"')
(2, '0.027*"fact" + 0.027*"integration" + 0.015*"establishment" + 0.015*"lobby" + 0.015*"agenda"')
(3, '0.030*"power" + 0.017*"rule" + 0.017*"idea" + 0.017*"even" + 0.017*"people"')
(4, '0.033*"member" + 0.033*"state" + 0.023*"create" + 0.012*"mandate" + 0.012*"responsibility"')
Thread 3
(0, '0.016*"trade" + 0.016*"course" + 0.016*"largely" + 0.016*"narrative" + 0.016*"medium"')
(1, '0.018*"brexit" + 0.013*"leave" + 0.013*"never" + 0.013*"job" + 0.013*"debate"')
(2, '0.013*"ultra" + 0.013*"brexit" + 0.013*"hard" + 0.013*"fact" + 0.013*"nowhere"')
(3, '0.031*"brexit" + 0.025*"would" + 0.019*"trade" + 0.019*"job" + 0.019*"leaver"')
(4, '0.029*"trade" + 0.015*"tory" + 0.015*"would" + 0.015*"market" + 0.015*"barrier"')
Thread 5
(0, '0.021*"help" + 0.012*"freedom" + 0.012*"make" + 0.012*"c

(2, '0.050*"candidate" + 0.039*"incumbent" + 0.038*"democratic" + 0.027*"businesswoman" + 0.026*"georgia"')
(3, '0.032*"candidate" + 0.032*"democratic" + 0.032*"2018" + 0.032*"vote" + 0.022*"attorney"')
(4, '0.032*"candidate" + 0.032*"democratic" + 0.027*"2018" + 0.027*"ga12" + 0.027*"ga07"')
Thread 19
(0, '0.018*"cette" + 0.018*"onoda" + 0.018*"vous" + 0.012*"donc" + 0.012*"après"')
(1, '0.018*"pour" + 0.018*"quelques" + 0.018*"onoda" + 0.012*"suzuki" + 0.012*"vous"')
(2, '0.021*"japonais" + 0.015*"février" + 0.015*"l’île" + 0.015*"débarque" + 0.015*"lubang"')
(3, '0.019*"dans" + 0.015*"onoda" + 0.015*"fait" + 0.015*"soldat" + 0.010*"plus"')
(4, '0.018*"mois" + 0.012*"même" + 0.012*"major" + 0.012*"donc" + 0.012*"japon"')
Thread 25
(0, '0.041*"say" + 0.021*"dean" + 0.021*"mother" + 0.021*"years" + 0.021*"father"')
(1, '0.020*"shot" + 0.020*"call" + 0.020*".223" + 0.020*"round" + 0.020*"getting"')
(2, '0.027*"time" + 0.015*"many" + 0.015*"face" + 0.015*"maniac" + 0.015*"mouth"')
(3, '0

(0, '0.022*"point" + 0.022*"dprk" + 0.015*"nation" + 0.015*"exist" + 0.015*"furthermore"')
(1, '0.041*"food" + 0.026*"dprk" + 0.021*"security" + 0.021*"government" + 0.021*"would"')
(2, '0.030*"dprk" + 0.030*"citizen" + 0.023*"food" + 0.016*"famine" + 0.016*"among"')
(3, '0.033*"military" + 0.023*"security" + 0.023*"food" + 0.023*"threat" + 0.023*"face"')
(4, '0.041*"dprk" + 0.018*"land" + 0.018*"food" + 0.013*"availability" + 0.013*"sanction"')
Thread 45
(0, '0.045*"master" + 0.031*"chair" + 0.031*"serve" + 0.017*"rothschild" + 0.017*"ask"')
(1, '0.034*"qanon" + 0.021*"release" + 0.014*"trace" + 0.014*"youtube" + 0.014*"shooter"')
(2, '0.034*"qanon" + 0.026*"pope" + 0.018*"aghdam" + 0.018*"police" + 0.018*"kill"')
(3, '0.030*"qanon" + 0.022*"manafort" + 0.021*"learn" + 0.021*"tell" + 0.020*"future"')
(4, '0.044*"qanon" + 0.020*"drop" + 0.020*"epstein" + 0.020*"tell" + 0.015*"next"')
Thread 46
(0, '0.031*"shearer" + 0.021*"opposition" + 0.021*"research" + 0.021*"kimberlin" + 0.012*"cod

(3, '0.027*"corn" + 0.020*"trump" + 0.020*"david" + 0.014*"source" + 0.014*"anti"')
(4, '0.025*"buzzfeed" + 0.017*"gubarev" + 0.017*"using" + 0.017*"case" + 0.009*"circle"')
Thread 65
(0, '0.014*"know" + 0.014*"think" + 0.014*"intelligence" + 0.014*"go" + 0.014*"really"')
(1, '0.045*"fusion" + 0.023*"mother" + 0.018*"politics" + 0.018*"trump" + 0.018*"jones"')
(2, '0.027*"dossier" + 0.018*"answer" + 0.018*"committee" + 0.018*"individual" + 0.010*"know"')
(3, '0.029*"fusion" + 0.015*"dossier" + 0.015*"hire" + 0.015*"2017" + 0.015*"circle"')
(4, '0.032*"fusion" + 0.022*"hire" + 0.022*"katsyv" + 0.022*"bruce" + 0.022*"information"')
Thread 66
(0, '0.037*"many" + 0.031*"playing" + 0.031*"thanks" + 0.031*"tweet" + 0.031*"around"')
(1, '0.111*"mueller" + 0.092*"answer" + 0.075*"guilty" + 0.075*"plea" + 0.056*"indictment"')
(2, '0.038*"quiz" + 0.038*"involvement" + 0.038*"campaign" + 0.038*"test" + 0.038*"russia"')
(3, '0.093*"investigation" + 0.075*"mueller" + 0.057*"days" + 0.039*"answer" +

(1, '0.093*"justice" + 0.093*"qanon" + 0.085*"fulldisclosure" + 0.085*"anon" + 0.085*"drop"')
(2, '0.057*"theory" + 0.057*"clock" + 0.057*"qanon" + 0.039*"followthepen" + 0.039*"follow"')
(3, '0.010*"disclosure" + 0.010*"full" + 0.010*"say" + 0.010*"green" + 0.010*"wray"')
(4, '0.057*"qanon" + 0.046*"say" + 0.046*"justice" + 0.035*"recuse" + 0.024*"brand"')
Thread 93
(0, '0.024*"hmmm" + 0.024*"right" + 0.024*"crash" + 0.024*"nuclear" + 0.024*"uranium"')
(1, '0.029*"...." + 0.029*"little" + 0.029*"risk" + 0.016*"wrong" + 0.016*"putin"')
(2, '0.050*"uranium" + 0.030*"mine" + 0.030*"rosatom" + 0.021*"license" + 0.021*"project"')
(3, '0.040*"uranium" + 0.040*"would" + 0.031*"rosatom" + 0.021*"putin" + 0.021*"sure"')
(4, '0.044*"uranium" + 0.038*"deal" + 0.025*"kazakhstan" + 0.019*"clinton" + 0.019*"think"')
Thread 100
(0, '0.023*"wilson" + 0.023*"like" + 0.023*"probably" + 0.023*"military" + 0.023*"scott"')
(1, '0.047*"anthem" + 0.047*"national" + 0.032*"song" + 0.032*"would" + 0.032*"play

### CSV twentyfive_thirty

In [25]:
THIS_FOLDER = os.getcwd()
threads_leer = threads5
carpeta_guardar = "tpcsv5"

#Poblar text_data

for hilos in threads_leer:
    camino = os.path.join(THIS_FOLDER, carpeta_guardar)
    text_data = []
    documentos = []
    dictionary = []
    corpus = []
    print(hilos)
    documentos = threads_leer[hilos]

    #print(documentos)

    for line in documentos:
        #print(line)
        tokens = prepare_text_for_lda(line)
        if random.random() > .009:
            #print(tokens)
            text_data.append(tokens)

    #print(text_data) 
    NDIC = camino+"\\"+hilos+"_t_dictionary1.gensim"
    NMOD = camino+"\\"+hilos+"_t_model1.gensim"
    NCOR = camino+"\\"+hilos+"_t_corpus1.pkl"
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open(NCOR, 'wb'))
    dictionary.save(NDIC)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(NMOD)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    for topic in topics:
        print(topic)

Thread 2
(0, '0.050*"data" + 0.036*"eunoia" + 0.022*"kogan" + 0.022*"agreement" + 0.015*"say"')
(1, '0.022*"data" + 0.022*"source" + 0.022*"eunoia" + 0.015*"2015" + 0.015*"delete"')
(2, '0.029*"wylie" + 0.018*"say" + 0.018*"eunoia" + 0.018*"idea" + 0.012*"palantir"')
(3, '0.028*"trump" + 0.017*"email" + 0.013*"eunoia" + 0.012*"campaign" + 0.012*"individual"')
(4, '0.047*"wylie" + 0.032*"eunoia" + 0.017*"lewandowski" + 0.012*"company" + 0.012*"pitch"')
Thread 5
(0, '0.043*"boomer" + 0.013*"even" + 0.013*"like" + 0.013*"think" + 0.013*"life"')
(1, '0.017*"life" + 0.017*"give" + 0.017*"work" + 0.017*"child" + 0.017*"every"')
(2, '0.038*"boomer" + 0.022*"going" + 0.017*"parent" + 0.012*"take" + 0.012*"another"')
(3, '0.025*"boomer" + 0.015*"people" + 0.015*"problem" + 0.015*"lack" + 0.015*"really"')
(4, '0.039*"boomer" + 0.020*"make" + 0.014*"people" + 0.014*"many" + 0.014*"back"')
Thread 6
(0, '0.027*"disneyprincess" + 0.021*"face" + 0.014*"split" + 0.014*"contract" + 0.014*"take"')
(1, '

(4, '0.005*"time" + 0.005*"incoming" + 0.005*"tension" + 0.005*"high" + 0.005*"ticking"')
Thread 28
(0, '0.022*"job" + 0.014*"support" + 0.014*"le" + 0.014*"white" + 0.009*"dominance"')
(1, '0.031*"white" + 0.020*"supremacy" + 0.016*"black" + 0.012*"social" + 0.012*"household"')
(2, '0.017*"saying" + 0.009*"even" + 0.009*"power" + 0.009*"group" + 0.009*"right"')
(3, '0.015*"white" + 0.015*"people" + 0.015*"perhaps" + 0.008*"sense" + 0.008*"problem"')
(4, '0.016*"woman" + 0.016*"white" + 0.012*"voter" + 0.008*"values" + 0.008*"traditional"')
Thread 29
(0, '0.032*"know" + 0.024*"people" + 0.017*"slightly" + 0.017*"dude" + 0.017*"use"')
(1, '0.048*"humint" + 0.020*"know" + 0.020*"collector" + 0.011*"tell" + 0.011*"network"')
(2, '0.027*"intelligence" + 0.019*"people" + 0.019*"sigint" + 0.019*"since" + 0.019*"dossier"')
(3, '0.016*"like" + 0.016*"weird" + 0.016*"file" + 0.016*"french" + 0.016*"folder"')
(4, '0.045*"steele" + 0.029*"dossier" + 0.023*"humint" + 0.018*"verify" + 0.018*"work"'

(3, '0.020*"qanon" + 0.015*"nxivm" + 0.015*"cult" + 0.010*"starting" + 0.010*"going"')
(4, '0.027*"elite" + 0.022*"woman" + 0.011*"circle" + 0.011*"member" + 0.011*"program"')
Thread 52
(0, '0.027*"people" + 0.021*"able" + 0.021*"stupid" + 0.014*"knowing" + 0.014*"need"')
(1, '0.019*"still" + 0.013*"campaign" + 0.013*"information" + 0.013*"leave" + 0.013*"would"')
(2, '0.031*"make" + 0.021*"need" + 0.021*"people" + 0.021*"language" + 0.012*"others"')
(3, '0.027*"horizon" + 0.021*"things" + 0.021*"going" + 0.014*"reach" + 0.014*"people"')
(4, '0.020*"know" + 0.020*"brexit" + 0.020*"issue" + 0.011*"shopping" + 0.011*"already"')
Thread 56
(0, '0.047*"belief" + 0.036*"justice" + 0.025*"man" + 0.025*"authority" + 0.025*"sjws"')
(1, '0.024*"intel" + 0.024*"keep" + 0.024*"secret" + 0.024*"determine" + 0.013*"social"')
(2, '0.041*"sjws" + 0.041*"christian" + 0.028*"man" + 0.015*"many" + 0.015*"people"')
(3, '0.046*"atheist" + 0.028*"lack" + 0.028*"empathy" + 0.019*"hope" + 0.019*"conservative"

(0, '0.208*"find" + 0.177*"vote" + 0.050*"info" + 0.035*"place" + 0.035*"poll"')
(1, '0.174*"find" + 0.113*"poll" + 0.100*"info" + 0.042*"drop" + 0.023*"call"')
(2, '0.062*"voting" + 0.062*"drop" + 0.062*"center" + 0.062*"washington" + 0.062*"connecticut"')
(3, '0.044*"poll" + 0.044*"info" + 0.044*"location" + 0.044*"michigan" + 0.044*"arizona"')
(4, '0.073*"place" + 0.040*"county" + 0.040*"location" + 0.040*"idaho" + 0.040*"clerk"')
Thread 70
(0, '0.014*"investigation" + 0.010*"iran" + 0.010*"call" + 0.010*"trump" + 0.010*"server"')
(1, '0.017*"even" + 0.017*"hillary" + 0.017*"using" + 0.012*"fusiongps" + 0.012*"include"')
(2, '0.015*"russia" + 0.015*"tweet" + 0.011*"hack" + 0.011*"qanon" + 0.011*"russian"')
(3, '0.015*"corruption" + 0.008*"treason" + 0.008*"expose" + 0.008*"massive" + 0.008*"uraniumone"')
(4, '0.015*"hillary" + 0.011*"deal" + 0.011*"russia" + 0.011*"corrupt" + 0.011*"destroy"')
Thread 74
(0, '0.021*"immigration" + 0.011*"better" + 0.011*"injustice" + 0.011*"could" + 

(4, '0.027*"marion" + 0.010*"come" + 0.010*"british" + 0.010*"along" + 0.010*"soldier"')


# Deteccion de topicos por threads

Al contrario del apartado anterior, se buscarán tópicos en el archivo completo, por lo que se considerará cada thread como un documento, para esto se unirán los tweets siendo considerados parrafos separados por saltos de linea "\n".


In [26]:
Tthreads1

['5. Create a path to a green card for E-2 investors. Include any children brought here before age 21.\n6. Let\'s really make a 10 year law: provisional green cards.\n7. Direct DHS to allow people to take the steps to correct their immigration status.\n8. Penalties besides deportation.\n24. Vote out elected officials with close ties to nativist, white nationalist, or fearmongering groups such as FAIR, CIS, NumbersUSA, US Inc., KKK, VDare, etc.\n25. Give immig judges true independence, more support staff.\n26. Backlog relief for India, China, Philippines, Mexico.\n15. Expand ESL instruction.\n16. Create state Offices of New Americans.\n17. Make real use of S visas to take down cartels.\n18. Make filing for citizenship free or almost free.\n19. Create path to green card for longterm TPS holders.\n20. Recognize "deportees" as a group for asylum.\nSo much can be done. Yet we\'re stuck between a border wall and DACA. And gutting protections for 90% of the currently undocumented. \n\nThey\'v

In [27]:
from gensim import corpora
import gensim
NUM_TOPICS = 20
NUM_WORDS = 10
import pickle

### CSV five_ten

In [28]:
THIS_FOLDER = os.getcwd()
threads_leer = Tthreads1
carpeta_guardar = "Ttpcsv1"

#Poblar text_data


camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)

#print(text_data) 
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.014*"trump" + 0.014*"maga" + 0.012*"qanon" + 0.011*"greatawakening" + 0.011*"followthewhiterabbit" + 0.011*"2018" + 0.011*"qposts" + 0.011*"save" + 0.011*"followsnowwhite" + 0.011*"wethepeople"')
(1, '0.013*"like" + 0.011*"cambridge" + 0.010*"candidate" + 0.009*"analytica" + 0.007*"héloïse" + 0.007*"mercer" + 0.006*"liberal" + 0.006*"hollywood" + 0.006*"many" + 0.006*"abélard"')
(2, '0.008*"would" + 0.007*"abortion" + 0.007*"need" + 0.006*"vous" + 0.006*"deepstate" + 0.006*"government" + 0.005*"say" + 0.005*"argument" + 0.005*"life" + 0.005*"pray"')
(3, '0.029*"trump" + 0.019*"cambridgeanalytica" + 0.016*"campaign" + 0.010*"election" + 0.010*"russia" + 0.009*"voter" + 0.008*"target" + 0.007*"data" + 0.007*"mercer" + 0.007*"schiff"')
(4, '0.009*"qanon" + 0.008*"seacrest" + 0.007*"deepstate" + 0.007*"abuse" + 0.007*"truth" + 0.007*"lawmaker" + 0.006*"say" + 0.005*"want" + 0.005*"know" + 0.005*"redpill"')
(5, '0.016*"well" + 0.010*"information" + 0.010*"facebook" + 0.010*"quality" 

### CSV Ten_fifteen

In [29]:
THIS_FOLDER = os.getcwd()
threads_leer = Tthreads2
carpeta_guardar = "Ttpcsv2"

#Poblar text_data


camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)

#print(text_data) 
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.019*"gitmo" + 0.010*"care" + 0.010*"thestorm" + 0.010*"greatawakening" + 0.010*"qanon" + 0.010*"sethrich" + 0.010*"releasethevideo" + 0.009*"people" + 0.008*"trump" + 0.007*"apollo"')
(1, '0.013*"post" + 0.010*"people" + 0.010*"disinformation" + 0.009*"warning" + 0.007*"agency" + 0.007*"like" + 0.006*"next" + 0.006*"anon" + 0.006*"look" + 0.004*"election"')
(2, '0.017*"trump" + 0.015*"cohen" + 0.012*"page" + 0.010*"stone" + 0.009*"grassleymemo" + 0.009*"never" + 0.008*"declassify" + 0.007*"campaign" + 0.006*"say" + 0.006*"portion"')
(3, '0.016*"cult" + 0.016*"nevergetourguns" + 0.009*"right" + 0.007*"shooting" + 0.007*"operation" + 0.007*"family" + 0.006*"child" + 0.006*"weapon" + 0.006*"know" + 0.006*"words"')
(4, '0.010*"libertarian" + 0.008*"privatize" + 0.008*"europe" + 0.007*"people" + 0.007*"country" + 0.007*"darpa" + 0.007*"qarmy" + 0.007*"wwg1wga" + 0.007*"really" + 0.006*"like"')
(5, '0.033*"qanon" + 0.011*"greatawakening" + 0.008*"thestorm" + 0.006*"time" + 0.006*"back

### CSV fifteen_twenty

In [30]:
THIS_FOLDER = os.getcwd()
threads_leer = Tthreads3
carpeta_guardar = "Ttpcsv3"

#Poblar text_data


camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)

#print(text_data) 
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.024*"qanon" + 0.015*"windsor" + 0.014*"greatawakening" + 0.013*"thestorm" + 0.012*"prince" + 0.012*"harry" + 0.012*"internetbillofrights" + 0.011*"meghan" + 0.010*"markle" + 0.010*"enjoytheshow"')
(1, '0.017*"pruitt" + 0.017*"bootpruitt" + 0.010*"times" + 0.009*"tesla" + 0.008*"page" + 0.008*"query" + 0.007*"month" + 0.007*"agent" + 0.006*"read" + 0.006*"make"')
(2, '0.027*"iran" + 0.013*"iranian" + 0.013*"nuclear" + 0.010*"sanction" + 0.008*"obama" + 0.005*"could" + 0.005*"program" + 0.005*"support" + 0.005*"enrichment" + 0.004*"reason"')
(3, '0.013*"click" + 0.009*"hindu" + 0.009*"list" + 0.009*"page" + 0.008*"muslim" + 0.008*"apps" + 0.007*"make" + 0.007*"fakenewsawards" + 0.006*"setting" + 0.006*"story"')
(4, '0.012*"2018mmm" + 0.012*"maddow" + 0.011*"water" + 0.010*"devil" + 0.010*"zwaan" + 0.009*"deer" + 0.009*"alex" + 0.008*"bank" + 0.008*"manafort" + 0.007*"mueller"')
(5, '0.011*"doubt" + 0.010*"jury" + 0.010*"innocent" + 0.010*"reasonable" + 0.009*"rape" + 0.009*"news" 

### CSV twenty_twentyfive

In [31]:
THIS_FOLDER = os.getcwd()
threads_leer = Tthreads4
carpeta_guardar = "Ttpcsv4"

#Poblar text_data


camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)

#print(text_data) 
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.063*"qanon" + 0.017*"drop" + 0.016*"justice" + 0.016*"anon" + 0.016*"post" + 0.015*"wwg1wgaa" + 0.013*"qalert" + 0.013*"4/11/18" + 0.011*"relate" + 0.011*"fulldisclosure"')
(1, '0.006*"people" + 0.006*"would" + 0.006*"trump" + 0.006*"woman" + 0.005*"dprk" + 0.005*"good" + 0.005*"kurd" + 0.005*"uranium" + 0.004*"could" + 0.004*"turkey"')
(2, '0.014*"profit" + 0.013*"fusion" + 0.010*"financial" + 0.009*"money" + 0.009*"system" + 0.008*"people" + 0.006*"dossier" + 0.005*"need" + 0.005*"know" + 0.005*"state"')
(3, '0.007*"state" + 0.006*"politico" + 0.006*"know" + 0.006*"burkman" + 0.006*"people" + 0.005*"want" + 0.005*"make" + 0.005*"take" + 0.004*"gates" + 0.004*"hazmat"')
(4, '0.016*"morgan" + 0.008*"black" + 0.006*"white" + 0.006*"time" + 0.005*"invent" + 0.005*"take" + 0.005*"country" + 0.004*"trump" + 0.004*"good" + 0.004*"tyranny"')
(5, '0.000*"wethepeople" + 0.000*"theresistance" + 0.000*"wwg1wga" + 0.000*"noconcon" + 0.000*"state" + 0.000*"qanon" + 0.000*"libertyrising" + 0

### CSV twentyfive_thirty

In [32]:
THIS_FOLDER = os.getcwd()
threads_leer = Tthreads5
carpeta_guardar = "Ttpcsv5"

#Poblar text_data


camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)

#print(text_data) 
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.020*"page" + 0.015*"bombshell" + 0.012*"russia" + 0.011*"statement" + 0.010*"trump" + 0.010*"false" + 0.009*"sessions" + 0.008*"campaign" + 0.007*"know" + 0.007*"evidence"')
(1, '0.009*"qanon" + 0.009*"internetbillofrights" + 0.008*"sungai" + 0.007*"iran" + 0.007*"empire" + 0.004*"committee" + 0.004*"congress" + 0.004*"yang" + 0.004*"information" + 0.004*"investigation"')
(2, '0.030*"iran" + 0.030*"iraq" + 0.014*"election" + 0.012*"iraqelections2018" + 0.012*"sadr" + 0.010*"coalition" + 0.009*"seats" + 0.009*"muqtada" + 0.009*"tehran" + 0.008*"party"')
(3, '0.021*"state" + 0.019*"illinois" + 0.013*"people" + 0.009*"could" + 0.007*"need" + 0.007*"general" + 0.007*"attorney" + 0.007*"campaign" + 0.006*"constitutional" + 0.006*"donation"')
(4, '0.011*"boomer" + 0.008*"people" + 0.007*"going" + 0.006*"time" + 0.006*"like" + 0.006*"back" + 0.006*"would" + 0.005*"tell" + 0.005*"could" + 0.005*"come"')
(5, '0.014*"people" + 0.009*"brexit" + 0.009*"getgee" + 0.009*"trump" + 0.009*"delet

## Megacorpus

Como tercera alternativa de análisis, se decide unir todos los threads que se tienen en un megacorpus, por lo que se utilizan cada thread de todos los archivos como un documento, luego se detectan los topicos presentes en los aproximadamente 500 documentos entregados.

In [33]:
megatexto = Tthreads1+Tthreads2+Tthreads3+Tthreads4+Tthreads5

In [34]:
#megatexto

['5. Create a path to a green card for E-2 investors. Include any children brought here before age 21.\n6. Let\'s really make a 10 year law: provisional green cards.\n7. Direct DHS to allow people to take the steps to correct their immigration status.\n8. Penalties besides deportation.\n24. Vote out elected officials with close ties to nativist, white nationalist, or fearmongering groups such as FAIR, CIS, NumbersUSA, US Inc., KKK, VDare, etc.\n25. Give immig judges true independence, more support staff.\n26. Backlog relief for India, China, Philippines, Mexico.\n15. Expand ESL instruction.\n16. Create state Offices of New Americans.\n17. Make real use of S visas to take down cartels.\n18. Make filing for citizenship free or almost free.\n19. Create path to green card for longterm TPS holders.\n20. Recognize "deportees" as a group for asylum.\nSo much can be done. Yet we\'re stuck between a border wall and DACA. And gutting protections for 90% of the currently undocumented. \n\nThey\'v

In [35]:
from gensim import corpora
import gensim
NUM_TOPICS = 20
NUM_WORDS = 10
import pickle

In [36]:
THIS_FOLDER = os.getcwd()
threads_leer = megatexto
carpeta_guardar = "mega"

#Poblar text_data

camino = os.path.join(THIS_FOLDER, carpeta_guardar)
text_data = []
documentos = []
dictionary = []
corpus = []
documentos = threads_leer

#print(documentos)

for line in documentos:
    #print(line)
    tokens = prepare_text_for_lda(line)
    if random.random() > .009:
        #print(tokens)
        text_data.append(tokens)
        
NDIC = camino+"\\t_dictionary1.gensim"
NMOD = camino+"\\t_model1.gensim"
NCOR = camino+"\\t_corpus1.pkl"
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open(NCOR, 'wb'))
dictionary.save(NDIC)

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save(NMOD)
topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.008*"qanon" + 0.007*"post" + 0.006*"maga" + 0.006*"people" + 0.005*"wwg1wga" + 0.005*"qalert" + 0.005*"yang" + 0.005*"4/4/18" + 0.005*"disneyprincess" + 0.004*"like"')
(1, '0.021*"2018" + 0.017*"democratic" + 0.016*"candidate" + 0.013*"vote" + 0.010*"incumbent" + 0.008*"register" + 0.008*"november" + 0.007*"midterms2018" + 0.007*"early" + 0.006*"voting"')
(2, '0.027*"qanon" + 0.010*"trump" + 0.008*"clinton" + 0.008*"post" + 0.006*"know" + 0.006*"say" + 0.005*"russia" + 0.005*"drop" + 0.004*"anon" + 0.004*"would"')
(3, '0.006*"work" + 0.006*"cambridge" + 0.005*"facebook" + 0.005*"need" + 0.005*"campaign" + 0.005*"analytica" + 0.004*"people" + 0.004*"leave" + 0.004*"data" + 0.004*"like"')
(4, '0.014*"qanon" + 0.013*"trump" + 0.007*"maga" + 0.005*"potus" + 0.005*"tuesdaythoughts" + 0.005*"post" + 0.005*"seth" + 0.005*"wwg1wga" + 0.005*"time" + 0.005*"tweet"')
(5, '0.011*"trump" + 0.007*"qanon" + 0.006*"maga" + 0.005*"people" + 0.005*"would" + 0.005*"know" + 0.005*"wwg1wga" + 0.004*

### Analisis de resultados de megacorpus

Luego de detectar los topicos, se clasificarán los threads de un archivo según los tópicos obtenidos.

In [37]:
for hilo in Tthreads1:
    hilito = prepare_text_for_lda(hilo)
    hilito_bow = dictionary.doc2bow(hilito)
    print(hilo)
    print(ldamodel.get_document_topics(hilito_bow))

5. Create a path to a green card for E-2 investors. Include any children brought here before age 21.
6. Let's really make a 10 year law: provisional green cards.
7. Direct DHS to allow people to take the steps to correct their immigration status.
8. Penalties besides deportation.
24. Vote out elected officials with close ties to nativist, white nationalist, or fearmongering groups such as FAIR, CIS, NumbersUSA, US Inc., KKK, VDare, etc.
25. Give immig judges true independence, more support staff.
26. Backlog relief for India, China, Philippines, Mexico.
15. Expand ESL instruction.
16. Create state Offices of New Americans.
17. Make real use of S visas to take down cartels.
18. Make filing for citizenship free or almost free.
19. Create path to green card for longterm TPS holders.
20. Recognize "deportees" as a group for asylum.
So much can be done. Yet we're stuck between a border wall and DACA. And gutting protections for 90% of the currently undocumented. 

They've been gutting it fo

I'm of the belief that the most successful politicians are the ones who will listen to the experts in whatever the respective field may be. Want to craft the best policy to deal with the opioid epidemic? Talk to the experts in that field. Don't shut them down.
[(5, 0.74336386), (19, 0.2495495)]
Pruitt is a danger to the public, yet is losing in the courts while conning @realDonaldTrump into saving his job...all while defying &amp; insulting his boss. Perhaps Trump is the empty vessel Scott Pruitt said he is. That would explain how Pruitt so easily conned him. #BootPruitt
REALITY CHECK: While Pruitt is a dangerous threat to the health of our families, his sloppy and careless legal work is undermining his ability to even implement his extreme pro-polluter agenda in several key cases. #BootPruitt https://t.co/lZKNC2kUjf
REALITY CHECK: Trump is also defending the guy who called him an "empty vessel." https://t.co/ovfQIUUssn #BootPruitt
REALITY CHECK: Pruitt has spent $3 MILLION on security

2/ Papadopoulos failing to live by his agreements with the feds/the court could lead to charges being brought forward that were previously withheld and/or contempt charges and/or reopening investigations otherwise on hold. None of the conditions Joyce said could be present exist.
[(15, 0.99173915)]
The real issue is, if the #CIA was doing these behavior and drug experiments in the 50s, #WTF makes us think they just decided to stop? 

They think we are sheep. 

They are wrong. @POTUS gave us back our voice.

#Qanon #TheStormHasArrived #MKUltra #RedPill https://t.co/rpHmPmIdW6
Stated by Senator Kennedy: “...there are perhaps any number of Americans who are walking around today ... who were given drugs with all the kinds of physical and psychological damage than can be caused.”[page 16]

#MKUltra #Qanon #TheStormHasArrived #CORRUPTION @POTUS https://t.co/qAkqYbEt98
Part of #MKUltra that led to its exposure was the death of Frank Olsen who was drugged with LSD by the CIA without his consen

5. It seems pretty clear CLOWNS control social media. Eric Schmidt (former Google CEO) likely wrote the code to censor across platforms. I'm guessing Russia, China, &amp; HK probably were sold this code by ES (or via HRC selling access to her unclass server?). https://t.co/rHiuN6p8JZ
[(6, 0.5679572), (13, 0.051863514), (19, 0.37435737)]
Startups should assume they'll get a ton of NOs from investors before a Yes. Identify partners with expertise &amp; track record in your industry, not just big names, and schedule meetings 3-4 weeks out. VCs are always on "vacation". 5/
If you don't hear back from an investor in 1-2 days, they don't like you. If they don't ask questions, they don't like you. Find one VC partner to be your champion and find out what their colleagues will want to know. 6/
Startup burn rates balloon to the size of your bank account if you're not careful. Be thrifty. Cash should last 12-18 months until you've hit milestones that de-risk your business for a new round of fund

(7) Put yourself in the chair. Masculinity is toxic. Republicans are uncompassionate. Trump is a Russian rape menace. How would you react to a world where #LoveTrumpsHate is compassion? Would you be confused? Would you snap? #mkultra https://t.co/dHi78r0ASZ
[(19, 0.9934483)]
+3DC/+3VA Eastern = 13,682 🔥SEALED Indictments
⚖️Dockets➡️ https://t.co/AIWErHTRGh 
🇷🇺#skolkovo➡️  https://t.co/tNKB1x6FPf
Btw 🇨🇳SD/WY/CO💰➡️https://t.co/NkAtbGbzFF  
 #QAnon #MemoWar @avery1776 @connieketchup @damartin32 @BabeReflex_8 @BasedBasterd @littlecarrotq https://t.co/QAcy2PD5q2
+8 DC/+8 VA Eastern = 16 NEW Sealed Indictments #FollowTheWhiteRabbit  #ReleaseTheMemo #SealedIndictments #TickTock  @connieketchup @almostjingo @damartin32 @littlecarrotq @BabeReflex_8 @passion_4truth 
🧐👇 
https://t.co/r4C3ern9Xd
Nationwide Tallies 
https://t.co/3xiTaFYEFi https://t.co/43xH9tbPMW
+1 DC/+0 VA Eastern =13,676 SEALED Indictments 
https://t.co/b1V490Y9MO
#FollowTheWhiteRabbit  #MemoWar #QAnon @connieketchup @damartin32

...

(7) The strongest argument for abortion on demand is the woman's bodily autonomy. Unfortunately for the left, that too fails, as soon as it's pointed out that half of babies are female. The argument for abortion after the age of viability (20 weeks) immediately fails, too.
[(19, 0.9948649)]
PS: just to confuse you guys even more, I have zero time for Jeff Sessions, but the man was pictured dining with Rosenstein for a reason, gang.

Steady as she goes, chaps. Hold your fire. We're getting to the end of this terrible period of US history now. Stay frosty. ;) https://t.co/nPehD1UiFb
If Andy McCabe opened a criminal investigation into Jeff Sessions but Mueller told Sessions lawyer that it had been closed, it was closed ***for a reason*** folks. And the reason wasn't "McCabe got it wrong, Sessions was transparent with Congress"
Jeff Sessions had no choice in the matter of firing McCabe, because of the Inspector General's report. 

I would caution everybody into reading too much into this. 

3/ If Bitcoin is to become a foundation for human and machine peer-to-peer interaction that nobody can control or censor, each user needs to easily be able to run a full node
[(0, 0.9912037)]
5. Anons are quick with the memes...lol. https://t.co/l1WWTU3Rpz
9. Love it when anons generate these graphics showing connections between the news and Q's drops. The  DS doesn't care who they kill to achieve their goal. They want to shut Q comms down. Must be over the target.

(h/t @M2Madness) https://t.co/Y56OtRw46T
2. Looks like Zuckerberg and RT (Rex Tillerson??) met w/8 others incl 1 fmr IC Dir (Brennan?) to strategize plan &amp; 4am Clown Media narrative/talking points. Q team still monitoring thanks to algorithm provided by @Snowden. Checkmate time? FB on last legs? Nobody gets free pass. https://t.co/5wkZGCswac
7. I agree w/these anons. The bombings were DS FF to highlight Q as dangerous "fake news". I had never heard the term "fake news" until right after @wikileaks published the Podesta 

So 'microtargeting' is just a means. This is the crucial difference people aren't getting. Even the data for the microtargeting (derived illicitly it seems) isn't as important as operating principle of #CambridgeAnalytica - it's not campaigning, it's a military grade weapon
[(15, 0.14678556), (17, 0.64590913), (19, 0.20233454)]
Cambridge Analytica's intensive psy ops campign "directly targeted Bernie Sanders voters to prevent them from voting" for Hillary Clinton -- and overran search engines, making real information harder to find. #FakeNews #Propaganda
In addition to the Cambridge Analytica campaign, Russia targeted the same voters with similar messaging that reached 126 million Americans #TrumpRussia #TrumpColluded https://t.co/9Te7xGoGtX
Undercover reporter: "So the candidate is the puppet?"

#CambridgeAnalytica CEO Alexander Nix: "Always." https://t.co/jYrZrZfek5
"In June 2016, the Trump campaign was flailing... the Mercers... offered Trump a huge cash injection, but they insisted

https://t.co/NzeX5VC6bX
[(2, 0.08665834), (3, 0.9087498)]
A corrupt media establishment is more dangerous —IMO— than having corrupt government. Hell, they ARE the government. Their silence on Julian’s torture tells all. 

#ReconnectJulian
You don’t have to like Julian. But know this: No one has come close to publishing what he has published, and NO ONE has EVER protected their sources like #Wikileaks has. Have you ever blown the whistle? If so, support him now. 

#ReconnectJulian
Thinking of @JulianAssange and how badly Hillary still wants him dead. 

Write to him.
“I was so ticked at him. Until I realized where he was headed..” #ReconnectJulian https://t.co/4bmNsbgz2G
Julian Assange has never had to retract a single story as #FakeNews. 

#Wikileaks has won every single court case that challenged their information. 

Every. Single. One. 

NO mainstream media network can ever say that. 

#ReconnectJulian
The publications Julian Assange has given us (undeservedly) have lifted the veil of

7/ The other variable I mentioned, stability of the network, got a massive boost in 2017, when users fought off corporate and political attacks. As a network, #bitcoin showed reliability and built trust that it won't be altered by insiders or govts.
[(6, 0.9953202)]
Par défaut, les personnes, pages et listes que vous suivez/likez SONT PUBLIQUES.
C'est à partir de ces informations que Cambridge Analytica semble avoir si bien fonctionné. (4/5)
Facebook explique que « vous pouvez contrôler la plupart des informations que d’autres [vos amis] peuvent communiquer à des applications » mais que « ces contrôles ne vous permettent pas de limiter l’accès à vos informations publiques et à la liste de vos amis ». (3/5)
https://t.co/amBueh5LWf :
« si vous publiez quelque chose sur Facebook, toute personne qui peut y accéder [vos amis] peut permettre à d’autres (comme des jeux, des applications ou des sites web qu’ils utilisent) d’y accéder » (2/5)
Facebook n'a pas obtenu votre consentement préalable

### Análisis de tópicos

Es posible analizar la relación entre los tópicos obtenidos a través de la librería pyLDAvis, la cual grafica la distancia entre los tópicos

In [38]:
dictionary = gensim.corpora.Dictionary.load(NDIC)
corpus = pickle.load(open(NCOR, 'rb'))
lda = gensim.models.ldamodel.LdaModel.load(NMOD)
import pyLDAvis.gensim

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\ProgramData\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:109: DeprecationWarning: invalid escape sequence \d
  nbits = re.compile('(\d+)bit').search(abits).group(1)
C:\ProgramData\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:662: DeprecationWarning: invalid escape sequence \s
  "\s+stepping\s+(?P<STP>\d+)", re.IGNORECASE)


# 